In [97]:
import pandas as pd
import numpy as np

import datetime as dt

from scipy.stats import ttest_ind, ttest_rel, ttest_1samp
from scipy.stats import f_oneway, pearsonr, chisquare, chi2_contingency
from scipy.stats import shapiro, levene, bartlett

In [98]:
# 데이터 불러오기
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e6_p1_1.csv')

df.head()

,구급보고서번호,소방서명,환자증상구분명1,신고일자,신고시각,출동일자,출동시각,환자직업명,사고발생장소
0,20211122103M01526,중랑소방서,심계항진,20210330,104800,20210330,105042,무직,집
1,20211111105M00771,노원소방서,복통,20210412,225600,20210412,225827,기타,집
2,20211115104M00864,서초소방서,심계항진,20210303,194100,20210303,194500,직장인,상업시설
3,20211113107M01869,마포소방서,고열,20211225,235400,20211226,331,유아,집
4,20211118102M05820,강동소방서,고열,20211019,12100,20211019,12322,무직,집


In [99]:
df['신고일자'] = df['신고일자'].astype('str')
df['신고시각'] = df['신고시각'].astype('str')

df['신고시각'] = df['신고시각'].map(lambda x:str(x).zfill(6))

df['신고_DATE'] = df['신고일자'] + df['신고시각']

print(df['신고_DATE'].head(10))

0    20210330104800
1    20210412225600
2    20210303194100
3    20211225235400
4    20211019012100
5    20210810122800
6    20211024091500
7    20210315235900
8    20210228091500
9    20210729124900
Name: 신고_DATE, dtype: object


In [100]:
df['출동일자'] = df['출동일자'].astype('str')
df['출동시각'] = df['출동시각'].astype('str')

df['출동시각'] = df['출동시각'].map(lambda x:str(x).zfill(6))

df['출동_DATE'] = df['출동일자'] + df['출동시각']

print(df['출동_DATE'].head(10))

0    20210330105042
1    20210412225827
2    20210303194500
3    20211226000331
4    20211019012322
5    20210810122900
6    20211024091725
7    20210316000239
8    20210228091900
9    20210729125146
Name: 출동_DATE, dtype: object


In [101]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2762 entries, 0 to 2761
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   구급보고서번호   2762 non-null   object
 1   소방서명      2762 non-null   object
 2   환자증상구분명1  2762 non-null   object
 3   신고일자      2762 non-null   object
 4   신고시각      2762 non-null   object
 5   출동일자      2762 non-null   object
 6   출동시각      2762 non-null   object
 7   환자직업명     2762 non-null   object
 8   사고발생장소    2762 non-null   object
 9   신고_DATE   2762 non-null   object
 10  출동_DATE   2762 non-null   object
dtypes: object(11)
memory usage: 237.5+ KB


##### 문제 1-1
- 각 구급보고서별 출동시각과 신고시각의 차이 -> '소요시간' 컬럼으로 추가 (초 단위)
- 소방서별 소요시간의 평균을 오름차순으로 정렬했을 때
- 3번째로 작은 소요시간 값과 소방서명을 출력하라

In [102]:
df['신고_DATE'] = pd.to_datetime(df['신고_DATE'])
df['출동_DATE'] = pd.to_datetime(df['출동_DATE'])

df['소요시각'] = (df['출동_DATE']-df['신고_DATE']).dt.total_seconds()

In [103]:
res = df.groupby(['소방서명'])['소요시각'].mean()
result = res.sort_values().reset_index()

print(result.iloc[2].values)

['종로소방서' 175.5]


In [104]:
# df['소요시각'] = (
#                 pd.to_datetime(
#                     df['출동일자'].astype('str') + df['출동시각'].astype('str').str.zfill(6)
#                 )

#                 - 
    
#                 pd.to_datetime(
#                     df['신고일자'].astype('str') + df['신고시각'].astype('str').str.zfill(6)
#                 )
#                 ).dt.total_seconds()


result = df.groupby(['소방서명'])['소요시각'].mean().sort_values().reset_index().iloc[2].values
print(result)

['종로소방서' 175.5]


In [105]:
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e6_p1_2.csv')

df.head()

,학교명,시도,학교세부유형,일반학급_학생수_계,교원수_총계_계
0,서울대학교사범대학부설고등학교,서울,일반고등학교,689,73
1,가락고등학교,서울,일반고등학교,765,70
2,개포고등학교,서울,일반고등학교,684,69
3,경기고등학교,서울,일반고등학교,1047,96
4,경동고등학교,서울,일반고등학교,529,58


##### 문제1-2

- 학교세부유형이 일반중학교인 학교들 중
- 일반중학교 숫자가 2번째로 많은 시도의 일반중학교 데이터만 필터
- 해당 시도의 교원 한명 당 맡은 학생수가 가장 많은 학교 찾기
- 해당 학교의 교원수 출력

In [106]:
cond1 = df['학교세부유형'] == '일반중학교'

print(df['학교세부유형'].value_counts())
print(df.info())

일반중학교     3260
일반고등학교    1700
Name: 학교세부유형, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4960 entries, 0 to 4959
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   학교명         4960 non-null   object
 1   시도          4960 non-null   object
 2   학교세부유형      4960 non-null   object
 3   일반학급_학생수_계  4960 non-null   int64 
 4   교원수_총계_계    4960 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 193.9+ KB
None


In [107]:
df1 = df[cond1]

df1.head()

,학교명,시도,학교세부유형,일반학급_학생수_계,교원수_총계_계
230,동원중학교,서울,일반중학교,379,36
231,면목중학교,서울,일반중학교,462,42
232,봉화중학교,서울,일반중학교,256,31
233,성일중학교,서울,일반중학교,313,30
234,신현중학교,서울,일반중학교,683,52


In [108]:
region = df1.groupby('시도').count().sort_values('학교세부유형', ascending=False).reset_index().iloc[1,0]

print(region)

서울


In [113]:
df2 = df.copy()

cond2 = df2['시도'] == region

df2['맡은학생수'] = df2['일반학급_학생수_계']/df2['교원수_총계_계']

df3 = df2[cond2].sort_values('맡은학생수', ascending=False)
teacher_num = df2[cond2].sort_values('맡은학생수', ascending=False).reset_index().loc[0,'교원수_총계_계']

# print(df3)
print(round(teacher_num))


33
